<a href="https://colab.research.google.com/github/jrhumberto/cd/blob/main/notebooks/Dinheiro_n%C3%A3o_tem_rastro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A análise dos dados de Financiamento de Campanha Eleitoral requer uma atenção redobrada em função do tipo de doação que foi destinada a cada um dos agentes partidários (AGP). Por exemplo, uma doação feita ao partido por uma pessoa física ou jurídica pode não ser encaminhada a um determinado candidato, ou seja, uma vez que a doação entra no caixa de um AGP, este não necessariamente será utilizado para um determinado candidato. Esta realidade se encaixa no ditado popular que diz “**dinheiro não tem rastro**”, ou seja, uma vez que o dinheiro entra em uma conta-corrente única (no caso a conta corrente de um agente partidário), perde-se o rastro de como ele será gasto ou transferido no futuro.

Para tentar rastrear o caminho do dinheiro, o TSE procurou meios de registrar as doações feitas indiretamente ao candidato a partir das doações feitas pelo AGP a um candidato, criando um atributo com a identificação do “Doador Original”, ou seja, a doação veio do partido mas o doador original foi uma pessoa física ou jurídica. Entretanto, fica a critério do AGP definir como será feito o rateio entre os candidatos de uma doação específica.

Ou seja, se o AGP não tem o interesse em deixar esse rastro evidente, essa informação se perde no emaranhado de transferências que os AGPs realizam durante a campanha eleitoral.

Esse notebook procura evidenciar um exemplo que mostra o caso acima narrado.

Os dados iniciais de teste são de 2014, quando a legislação ainda permitia doações de Pessoas Jurídicas. Os dados estão vindo da base de financiamento copiadas para o servidor do FARMI. Caso você queira validar os dados, basta copiar estes mesmos arquivos para o seu ambiente do Google Colab.

Caso você tenha alguma dúvida ou consideração, envie um email pra trocarmos uma ideia: josircg@gmail.com

A primeira consulta vai na base de doações feitas diretamente ao partido.

In [ ]:
import io
import csv
import requests

count = 0
total = 0
response = requests.get('http://www.farmi.pro.br/media/uploads/financiamento/receitas_partidos_2014_brasil.txt')
for registro in csv.DictReader(io.StringIO(response.text), delimiter=';'):
    partido = registro['Sigla  Partido']
    fonte = registro['Tipo receita']
    valor = float(registro['Valor receita'].replace(',','.'))
    doador = registro['CPF/CNPJ do doador originário'].strip()
    if doador == '#NULO':
        doador = registro['CPF/CNPJ do doador'].strip()
    
    # if fonte in ('Recursos de pessoas físicas','Recursos de pessoas jurídicas') and \
    if doador == '48540421000131' and registro['Sigla  Partido'] == 'PT':
        # print(fonte)
        print('%s: %s -> %s: %s' % (registro['Data da receita'], registro['Nome do doador'], registro['UF'], valor))        
        count += 1
        total += valor
print('Total: %s' % total)

12-SEP-14: Direção Nacional -> AC: 95000.0
09-OCT-14: SERVENG CIVILSAN S A EMPRESAS ASSOCIADAS DE ENGENHARIA -> BA: 1500000.0
30-SEP-14: SERVENG CIVILSAN SA EMPRESAS ASSOCIADAS DE ENGENHARIA -> SC: 100000.0
05-SEP-14: SERVENG CIVILSAN SA EMPRESAS ASSOCIADAS DE ENGENHARIA -> SC: 200000.0
27-AUG-14: SERVENG CIVILSAN SA EMPRESAS ASSOCIADAS DE ENGENHARIA -> SC: 200000.0
02-OCT-14: SERVENG CIVILSAN S A EMPRESAS ASSOCIADAS DE ENGENHARIA -> BR: 750000.0
12-SEP-14: SERVENG CIVILSAN S A EMPRESAS ASSOCIADAS DE ENGENHARIA -> BR: 550000.0
Total: 3395000.0


O resultado indica 7 doações da empresa para a Direção Nacional e que estas foram repassadas para os diretórios estaduais. Reparem entretanto que o total final está *errado* pois uma das doações foi um repasse para o Acre, ou seja, o valor total doado pela empresa **não** pode considerar a transferência da Direção Nacional para o Diretório do Acre!

Ou seja, não basta apenas somar os valores. Deve-se interpretar a transação realizada.

Vamos agora ao segundo arquivo que contém as transferências entre diretórios estaduais, municipais e comitês eleitorais.

In [ ]:
count = 0
response = requests.get('http://www.farmi.pro.br/media/uploads/financiamento/receitas_comites_2014_brasil.txt')
for registro in csv.DictReader(io.StringIO(response.text), delimiter=';'):
    partido = registro['Sigla  Partido']
    fonte = registro['Tipo receita']
    valor = float(registro['Valor receita'].replace(',','.'))
    doador = registro['CPF/CNPJ do doador originário'].strip()
    if doador == '#NULO':
        doador = registro['CPF/CNPJ do doador'].strip()
    if doador == '48540421000131' and partido == 'PT':
        # print(fonte)
        print('%s: %s -> %s(%s): %s' % (registro['Data da receita'], registro['Nome do doador'], registro['Tipo Comite'], registro['UF'], valor))
        count += 1

15/09/201400:00:00: Direção Estadual/Distrital -> Comitê Financeiro Distrital/Estadual para Governador(AC): 44000.0
11/09/201400:00:00: ELEIÇÃO 2014 FERNANDO DAMATA PIMETEL GOVERNADOR -> Comitê Financeiro Único(MG): 500000.0


Encontramos nesse arquivo 2 movimentações: uma do diretório do Acre para o comitê financeiro do Governador e uma movimentação da campanha do candidato a governador para o comitê central do Estado de Minas.

Reparem que apenas parte da doação de 95.000 recebida pelo diretório do Acre é repassada para o governador. Ou seja, o diretório continuou com 51.000 em caixa.

Algumas considerações: 

1) na base de candidatos, não se pode considerar uma doação feita pelo Diretório Nacional ou Federal como uma doação do partido para o candidato. Deve-se sempre verificar o doador original e considerar nesse caso que a doação veio na verdade de uma pessoa jurídica ou física.

2) Para uma análise fina, é sempre é interessante consultar a SPCE (Sistema de Prestação de Contas Eleitorais). É uma forma de conferir se os dados analisados estão compatíveis com os relatórios do TSE.

Vamos fazer a verificação no arquivo maior das doações dos candidatos:

In [ ]:
count = 0
response = requests.get('http://www.farmi.pro.br/media/uploads/financiamento/receitas_candidatos_2014_brasil.txt')
for registro in csv.DictReader(io.StringIO(response.text), delimiter=';'):
    partido = registro['Sigla  Partido']
    fonte = registro['Tipo receita']
    valor = float(registro['Valor receita'].replace(',','.'))
    doador = registro['CPF/CNPJ do doador originário'].strip()
    if doador == '#NULO':
        doador = registro['CPF/CNPJ do doador'].strip()    
    # if registro['CPF/CNPJ do doador'] != registro['CPF/CNPJ do doador originário'] and \
    if doador == '48540421000131' and registro['Sigla  Partido'] == 'PT':
        print('%s (%s) -> %s: %s (%s)' % (registro['Nome do doador'], registro['UF'], 
                                          registro['Nome candidato'], valor, fonte))
        count += 1

ELEIÇÃO 2014 ADRIANO DIOGO DEPUTADO FEDERAL (SP) -> JOSÉ PRADO DE ANDRADE: 13600.0 (Recursos de outros candidatos/comitês)
ELEIÇÃO 2014 JOSÉ VIEIRA FILHO (MG) -> GABRIEL GUIMARÃES DE ANDRADE: 243.0 (Recursos de outros candidatos/comitês)
Direção Nacional (MG) -> GABRIEL GUIMARÃES DE ANDRADE: 75000.0 (Recursos de partido político)
Direção Nacional (SP) -> LUIZ CLAUDIO MARCOLINO: 190000.0 (Recursos de partido político)
Direção Nacional (RJ) -> EDSON SANTOS DE SOUZA: 95000.0 (Recursos de partido político)
Direção Nacional (SP) -> ARLINDO CHIGNALIA JUNIOR: 237500.0 (Recursos de partido político)
Direção Nacional (SP) -> CANDIDO ELPIDIO DE SOUZA VACCAREZZA: 237500.0 (Recursos de partido político)
Direção Nacional (PA) -> PAULO ROBERTO GALVAO DA ROCHA: 37500.0 (Recursos de partido político)
Direção Nacional (SP) -> ADRIANO DIOGO: 95000.0 (Recursos de partido político)
ELEIÇÃO 2014 ARLINDO CHINAGLIA JUNIOR DEPUTADO FEDERAL (SP) -> JOÃO PAULO RILLO: 50000.0 (Recursos de outros candidatos/comit

Reparem que não existe referência de nenhuma doação da campanha do governador do Acre indicando que houve uma doação da referida empresa, ou seja, apesar da empresa ter doado para a campanha, ela não aparece como doadora da campanha do governador ou de qualquer outro candidato do PT no Acre!

# Visualização Gráfica

Vamos agora tentar visualizar estes mesmos dados graficamente. Para tal utilizaremos o tipo de gráfico Sanken.

Para tal, montaremos um grafo com os principais atores destas movimentações:

* PJ: Pessoa Jurídica, ou seja, a empresa que está fazendo a doação
* BR: O diretório nacional do Partido
* UF: A unidade da federação do Diretório Estadual ou Municipal
* COM-UF: O conjunto de comitês de um determinada UF
* CAN: O conjunto de candidatos

In [ ]:
grafo = {}

def add_node(origem, destino, valor):
    if origem != destino:
        chave = origem+'/'+destino
        if chave not in grafo:
            grafo[chave] = {'origem':origem, 'destino':destino, 'total':0 }
        grafo[chave]['total'] += valor

In [ ]:
import csv
import io
import requests

count = 0
response = requests.get('http://www.farmi.pro.br/media/uploads/financiamento/receitas_partidos_2014_brasil.txt')
for registro in csv.DictReader(io.StringIO(response.text), delimiter=';'):
    count += 1
    partido = registro['Sigla  Partido']
    fonte = registro['Tipo receita']
    valor = float(registro['Valor receita'].replace(',','.'))
    doador = registro['CPF/CNPJ do doador originário'].strip()
    if doador == '#NULO':
        doador = registro['CPF/CNPJ do doador'].strip()
        direto = True
    else:
        direto = False
    
    # if fonte in ('Recursos de pessoas físicas','Recursos de pessoas jurídicas') and \
    if doador == '48540421000131' and registro['Sigla  Partido'] == 'PT':
        print('%s: %s -> %s: %s' % (registro['Data da receita'], registro['Nome do doador'], registro['UF'], valor))
        if direto:
            origem = 'PJ'
            destino = registro['UF']
        else:
            origem = registro['Sigla UE doador'] 
            destino = registro['UF']
        add_node(origem, destino, valor)

print('Total de registros lidos: %d' % count)

12-SEP-14: Direção Nacional -> AC: 95000.0
09-OCT-14: SERVENG CIVILSAN S A EMPRESAS ASSOCIADAS DE ENGENHARIA -> BA: 1500000.0
30-SEP-14: SERVENG CIVILSAN SA EMPRESAS ASSOCIADAS DE ENGENHARIA -> SC: 100000.0
05-SEP-14: SERVENG CIVILSAN SA EMPRESAS ASSOCIADAS DE ENGENHARIA -> SC: 200000.0
27-AUG-14: SERVENG CIVILSAN SA EMPRESAS ASSOCIADAS DE ENGENHARIA -> SC: 200000.0
02-OCT-14: SERVENG CIVILSAN S A EMPRESAS ASSOCIADAS DE ENGENHARIA -> BR: 750000.0
12-SEP-14: SERVENG CIVILSAN S A EMPRESAS ASSOCIADAS DE ENGENHARIA -> BR: 550000.0
Total de registros lidos: 12247


In [ ]:
# Como ficou o grafo
for item in grafo.values():
    print(item)

{'origem': 'BR', 'destino': 'AC', 'total': 95000.0}
{'origem': 'PJ', 'destino': 'BA', 'total': 1500000.0}
{'origem': 'PJ', 'destino': 'SC', 'total': 500000.0}
{'origem': 'PJ', 'destino': 'BR', 'total': 1300000.0}


In [ ]:
# Importação das movimentações dos comitês e diretórios

count = 0
response = requests.get('http://www.farmi.pro.br/media/uploads/financiamento/receitas_comites_2014_brasil.txt')
for registro in csv.DictReader(io.StringIO(response.text), delimiter=';'):
    count += 1
    partido = registro['Sigla  Partido']
    fonte = registro['Tipo receita']
    valor = float(registro['Valor receita'].replace(',','.'))
    doador = registro['CPF/CNPJ do doador originário'].strip()
    if doador == '#NULO':
        doador = registro['CPF/CNPJ do doador'].strip()
    if doador == '48540421000131' and partido == 'PT':
        print('%s: %s -> %s(%s): %s' % (registro['Data da receita'], registro['Nome do doador'], 
                                   registro['Tipo Comite'], registro['UF'], valor))
        if fonte == 'Recursos de outros candidatos/comitês':
            origem = 'CAN'
        else:
            origem = registro['Sigla UE doador']
        if registro['Tipo Comite'].find('Comitê') != -1:
            destino = 'COM-'+registro['UF']
        else:
            destino = registro['UF']
        add_node(origem,destino,valor)
        print(origem,destino)

print('Total de registros lidos: %d' % count)

15/09/201400:00:00: Direção Estadual/Distrital -> Comitê Financeiro Distrital/Estadual para Governador(AC): 44000.0
AC COM-AC
11/09/201400:00:00: ELEIÇÃO 2014 FERNANDO DAMATA PIMETEL GOVERNADOR -> Comitê Financeiro Único(MG): 500000.0
CAN COM-MG
Total de registros lidos: 9173


In [ ]:
count = 0
response = requests.get('http://www.farmi.pro.br/media/uploads/financiamento/receitas_candidatos_2014_brasil.txt')
for registro in csv.DictReader(io.StringIO(response.text), delimiter=';'):
    count += 1
    partido = registro['Sigla  Partido']
    fonte = registro['Tipo receita']
    valor = float(registro['Valor receita'].replace(',','.'))
    doador = registro['CPF/CNPJ do doador originário'].strip()
    if doador == '#NULO':
        doador = registro['CPF/CNPJ do doador'].strip()    
    # if registro['CPF/CNPJ do doador'] != registro['CPF/CNPJ do doador originário'] and \
    if doador == '48540421000131' and registro['Sigla  Partido'] == 'PT':
        print('%s (%s) -> %s: %s (%s)' % (registro['Nome do doador'], registro['Sigla UE doador'], 
                                          registro['Nome candidato'], valor, fonte))
        if fonte == 'Recursos de partido político':
            origem = registro['Sigla UE doador']

        elif fonte == 'Recursos de outros candidatos/comitês':
            if registro['Nome do doador'].find('Comitê') != -1:
                origem = 'COM-'+registro['Sigla UE doador']
            else:
                origem = 'CAN'
        else:
            origem = 'PJ'
        destino = 'CAN'
        if origem != destino:
            add_node(origem, destino, valor)
            print('%s > %s' % (origem, destino))

print('Total de registros lidos: %d' % count)

ELEIÇÃO 2014 ADRIANO DIOGO DEPUTADO FEDERAL (SP) -> JOSÉ PRADO DE ANDRADE: 13600.0 (Recursos de outros candidatos/comitês)
ELEIÇÃO 2014 JOSÉ VIEIRA FILHO (MG) -> GABRIEL GUIMARÃES DE ANDRADE: 243.0 (Recursos de outros candidatos/comitês)
Direção Nacional (BR) -> GABRIEL GUIMARÃES DE ANDRADE: 75000.0 (Recursos de partido político)
BR > CAN
Direção Nacional (BR) -> LUIZ CLAUDIO MARCOLINO: 190000.0 (Recursos de partido político)
BR > CAN
Direção Nacional (BR) -> EDSON SANTOS DE SOUZA: 95000.0 (Recursos de partido político)
BR > CAN
Direção Nacional (BR) -> ARLINDO CHIGNALIA JUNIOR: 237500.0 (Recursos de partido político)
BR > CAN
Direção Nacional (BR) -> CANDIDO ELPIDIO DE SOUZA VACCAREZZA: 237500.0 (Recursos de partido político)
BR > CAN
Direção Nacional (BR) -> PAULO ROBERTO GALVAO DA ROCHA: 37500.0 (Recursos de partido político)
BR > CAN
Direção Nacional (BR) -> ADRIANO DIOGO: 95000.0 (Recursos de partido político)
BR > CAN
ELEIÇÃO 2014 ARLINDO CHINAGLIA JUNIOR DEPUTADO FEDERAL (SP) ->

In [ ]:
for item in grafo.values():
    print(item)

{'origem': 'BR', 'destino': 'AC', 'total': 95000.0}
{'origem': 'PJ', 'destino': 'BA', 'total': 1500000.0}
{'origem': 'PJ', 'destino': 'SC', 'total': 500000.0}
{'origem': 'PJ', 'destino': 'BR', 'total': 1300000.0}
{'origem': 'AC', 'destino': 'COM-AC', 'total': 44000.0}
{'origem': 'CAN', 'destino': 'COM-MG', 'total': 500000.0}
{'origem': 'BR', 'destino': 'CAN', 'total': 2967500.0}
{'origem': 'BA', 'destino': 'CAN', 'total': 1503040.5799999998}
{'origem': 'DF', 'destino': 'CAN', 'total': 1410000.0}
{'origem': 'SC', 'destino': 'CAN', 'total': 500000.0}
{'origem': 'COM-MG', 'destino': 'CAN', 'total': 512156.53}
{'origem': 'PJ', 'destino': 'CAN', 'total': 633000.0}


In [ ]:
!pip install plotly

In [ ]:
# Primeiro, vamos montar a lista de atores (nós do gráfico)
# Fixando o primeiro como a empresa (PJ)
count_atores = 0
atores = {'PJ': count_atores}
labels = ['PJ']
origem = []
destino = []
values = []
for chave in grafo:
    r = grafo[chave]
    index_origem = atores.get(r['origem'], None)
    if index_origem is None:
        count_atores += 1
        index_origem = count_atores
        atores[r['origem']] = index_origem
        labels.append(r['origem'])

    index_destino = atores.get(r['destino'], None)
    if index_destino is None:
        count_atores += 1
        index_destino = count_atores
        atores[r['destino']] = index_destino
        labels.append(r['destino'])
    
    origem.append( index_origem )
    destino.append( index_destino )
    values.append( int(r['total']) )
    
print('Total de Nós:%d' % (count_atores+1))

Total de Nós:9


In [ ]:
# Por fim, tentamos mostrar graficamente a movimentação das doações feitas pela empresa entre os atores

import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 20,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = labels,
      color = "blue"
    ),
    link = dict(
      source = origem,
      target = destino,
      value = values
  ))])

fig.update_layout(title_text="Doações do CNPJ 48.540.421/0001-31 em 2014 para o PT", font_size=12)
fig.show()

Problemas encontrados:

1) O Diretório do DF indicou que doações feitas a candidatos tiveram origem de doações da empresa. Entretanto não houve nenhuma doação da empresa ou do Diretório Nacional para o Diretório do DF!

2) A empresa doou 1,3 milhões ao Diretório Nacional (BR) mas o mesmo diretório indicou que houveram doações aos candidatos originários da empresa por um valor maior (2,97M)!

Ou seja, tem algo que não está batendo nessa análise. Vamos rever as doações do DF para tentar encontrar o problema.

In [ ]:
import csv
import io
import requests

count = 0
response = requests.get('http://www.farmi.pro.br/media/uploads/financiamento/receitas_candidatos_2014_brasil.txt')
for registro in csv.DictReader(io.StringIO(response.text), delimiter=';'):
    partido = registro['Sigla  Partido']
    fonte = registro['Tipo receita']
    valor = float(registro['Valor receita'].replace(',','.'))
    if registro['Numero Recibo Eleitoral'] == '000130300000DF002045': # Caso Agnelo
        break

registro

OrderedDict([('Cód. Eleição', '143'),
             ('Desc. Eleição', 'Eleições Gerais 2014'),
             ('Data e hora', '09/07/201617:15:01'),
             ('CNPJ Prestador Conta', '20574235000102'),
             ('Sequencial Candidato', '70000000818'),
             ('UF', 'DF'),
             ('Sigla  Partido', 'PT'),
             ('Numero candidato', '13'),
             ('Cargo', 'Governador'),
             ('Nome candidato', 'AGNELO SANTOS QUEIROZ FILHO'),
             ('CPF do candidato', '19667655504'),
             ('Numero Recibo Eleitoral', '000130300000DF002045'),
             ('Numero do documento', '850503'),
             ('CPF/CNPJ do doador', '00676213000138'),
             ('Nome do doador', 'Direção Nacional'),
             ('Nome do doador (Receita Federal)',
              'PARTIDO DO MOVIMENTO DEMOCRATICO BRASILEIRO - DIRETORIO NACIONAL'),
             ('Sigla UE doador', 'BR'),
             ('Número partido doador', '15'),
             ('Número candidato doador', '1

Reparem no atributo 'Número partido doador'. Essa doação não veio da Diretório Estadual do PT. **Veio do PMDB!** Ou seja, muitas doações são intra-partidos, ou seja, um partido envia a doação para outro partido.

Teremos que refazer o grafo considerando outros partidos como novos nós na rede.

O primeiro problema para resolver a equação é realizar um tratamento de dados pois o arquivo do TSE não informa a Sigla do Partido Doador e sim o número do partido. Dessa forma, teremos que fazer uma correção entre a sigla e o número.

In [ ]:
# Montar a lista de numeração dos partidos
# Essa lista é necessária pois a tabela de doações de partido não contém o número do partido recebedor (necessário para que possamos comparar com o número do partido doador)
import io
import csv
import requests

partidos = {}
response = requests.get('http://www.farmi.pro.br/media/uploads/financiamento/receitas_candidatos_2014_brasil.txt')
for registro in csv.DictReader(io.StringIO(response.text), delimiter=';'):
    partido = registro['Sigla  Partido']
    numero = registro['Numero candidato'][:2]
    if partido in partidos:
        if partidos[partido] != numero:
            print('Inconsistência %s atual:%s novo:%s' % (partido, partidos[partido], numero) )
    else:
        partidos[partido] = numero
        
# Cria índice invertido (para encontrar a sigla a partir do seu número)
partidos_numero = {}
for item in partidos:
    partidos_numero[ partidos[item] ] = item

for item in partidos:
    print(item, partidos[item])

PSDB 45
PSD 55
PSB 40
PMDB 15
PRB 10
PT 13
PPL 54
PTB 14
PP 11
PROS 90
PSOL 50
PPS 23
PV 43
PR 22
PC do B 65
PDT 12
PMN 33
PRP 44
PSC 20
DEM 25
PT do B 70
PRTB 28
PEN 51
SD 77
PHS 31
PSL 17
PTN 19
PTC 36
PSDC 27
PSTU 16
PCB 21
PCO 29


Vamos estender a consulta, estendendo o grafo para indicar também os partidos.

A codificação do autores ficará da seguinte forma:

* PJ: Pessoa Jurídica, ou seja, a empresa que está fazendo a doação
* Sigla do Partido-BR: O diretório nacional do Partido
* Sigla do Partido-UF: A unidade da federação do Diretório Estadual ou Municipal
* CAN: O conjunto de candidatos

Neste estudo, iremos agrupar os diretórios estaduais e comissões em um mesmo nó para facilitar o entedimento e desenho do gráfico.

Vamos refazer o grafo para os 3 arquivos: Partido, Comitê e Candidato:

In [ ]:
# Próximo passo é zerar o grafo e refazê-lo considerando as movimentações entre partido.
# Repare que a busca continua sendo apenas para as doações feitas ao PT mas agora especificamos de qual partido veio a doação.

grafo = {}

import csv

count = 0
partidos_diferentes = 0
response = requests.get('http://www.farmi.pro.br/media/uploads/financiamento/receitas_partidos_2014_brasil.txt')
for registro in csv.DictReader(io.StringIO(response.text), delimiter=';'):
    count += 1    
    partido = registro['Sigla  Partido']
    fonte = registro['Tipo receita']
    valor = float(registro['Valor receita'].replace(',','.'))
    doador = registro['CPF/CNPJ do doador originário'].strip()
    if doador == '#NULO':
        doador = registro['CPF/CNPJ do doador'].strip()
        direto = True
    else:
        direto = False
        
    numero_origem = registro['Número partido doador']
    numero_destino = partidos[ partido ]
    if numero_origem != '#NULO' and numero_origem != numero_destino:
        partidos_diferentes += 1
        partido_origem = partidos_numero[ numero_origem ]
    else:
        partido_origem = partido
    
    # if fonte in ('Recursos de pessoas físicas','Recursos de pessoas jurídicas') and \
    if doador == '48540421000131':
        print('%s: %s -> %s: %s' % (registro['Data da receita'], registro['Nome do doador'], registro['UF'], valor))
        if direto:
            origem = 'PJ'
            destino = partido+'-'+registro['UF']
        else:
            origem = partido_origem+'-'+registro['Sigla UE doador']
            destino = partido+'-'+registro['UF']
        add_node(origem, destino, valor)
        
print('Total de Registros: %d' % count)
print('Total de Doações entre partidos: %d' % partidos_diferentes)
print('Total de Nós registrados: %d' % len(grafo))

26-AUG-14: Comitê Financeiro Distrital/Estadual para Governador -> BR: 145.0
12-SEP-14: Direção Nacional -> AC: 95000.0
02-OCT-14: Direção Nacional -> BR: 237500.0
02-OCT-14: Direção Nacional -> AL: 400000.0
18-SEP-14: Direção Nacional -> DF: 420000.0
29-JUL-14: Direção Nacional -> DF: 1000000.0
09-OCT-14: SERVENG CIVILSAN S A EMPRESAS ASSOCIADAS DE ENGENHARIA -> BA: 1500000.0
30-SEP-14: SERVENG CIVILSAN SA EMPRESAS ASSOCIADAS DE ENGENHARIA -> SC: 100000.0
05-SEP-14: SERVENG CIVILSAN SA EMPRESAS ASSOCIADAS DE ENGENHARIA -> SC: 200000.0
27-AUG-14: SERVENG CIVILSAN SA EMPRESAS ASSOCIADAS DE ENGENHARIA -> SC: 200000.0
02-OCT-14: SERVENG CIVILSAN S A EMPRESAS ASSOCIADAS DE ENGENHARIA -> BR: 750000.0
12-SEP-14: SERVENG CIVILSAN S A EMPRESAS ASSOCIADAS DE ENGENHARIA -> BR: 550000.0
25-JUL-14: SERVENG CIVILSAN S A EMPRESAS ASSOCIADAS DE ENGENHARIA  -> BR: 1000000.0
12-SEP-14: SERVENG CIVILSAN S A EMPRESAS ASSOCIADAS DE ENGENHARIA  -> BR: 420000.0
30-SEP-14: SERVENG CIVILSAN S A EMPRESAS ASSOC

In [ ]:
for item in grafo.values():
    print(item)

{'origem': 'PSDB-SP', 'destino': 'PTN-BR', 'total': 145.0}
{'origem': 'PT-BR', 'destino': 'PT-AC', 'total': 95000.0}
{'origem': 'PT-BR', 'destino': 'PC do B-BR', 'total': 237500.0}
{'origem': 'PMDB-BR', 'destino': 'PMDB-AL', 'total': 400000.0}
{'origem': 'PMDB-BR', 'destino': 'PMDB-DF', 'total': 1420000.0}
{'origem': 'PJ', 'destino': 'PT-BA', 'total': 1500000.0}
{'origem': 'PJ', 'destino': 'PT-SC', 'total': 500000.0}
{'origem': 'PJ', 'destino': 'PT-BR', 'total': 1300000.0}
{'origem': 'PJ', 'destino': 'PMDB-BR', 'total': 4220000.0}
{'origem': 'PJ', 'destino': 'PSC-MG', 'total': 20000.0}
{'origem': 'PJ', 'destino': 'PR-BR', 'total': 100000.0}
{'origem': 'PJ', 'destino': 'DEM-SP', 'total': 200000.0}
{'origem': 'PJ', 'destino': 'DEM-DF', 'total': 100000.0}
{'origem': 'PJ', 'destino': 'PSB-SC', 'total': 500000.0}
{'origem': 'PJ', 'destino': 'PSB-BR', 'total': 500000.0}
{'origem': 'PJ', 'destino': 'PSDB-SP', 'total': 3250000.0}


In [ ]:
import csv
import io
import requests

count = 0
response = requests.get('http://www.farmi.pro.br/media/uploads/financiamento/receitas_comites_2014_brasil.txt')
for registro in csv.DictReader(io.StringIO(response.text), delimiter=';'):
    partido = registro['Sigla  Partido']
    fonte = registro['Tipo receita']
    valor = float(registro['Valor receita'].replace(',','.'))
    doador = registro['CPF/CNPJ do doador originário'].strip()
    if doador == '#NULO':
        doador = registro['CPF/CNPJ do doador'].strip()
    else:
        nome_doador = registro['Nome do doador originário']
        
    numero_origem = registro['Número partido doador']
    numero_destino = partidos[ partido ]
    if numero_origem != '#NULO' and numero_origem != numero_destino:
        partidos_diferentes += 1
        partido_origem = partidos_numero[ numero_origem ]
    else:
        partido_origem = partido

    if fonte == 'Recursos de pessoas jurídicas':
        origem = 'PJ'
        nome_doador = registro['Nome do doador']
    else:
        nome_doador = '%s (%s-%s)' % (
            registro['Nome do doador'].strip(), 
            partido_origem, 
            registro['Sigla UE doador'])
        if fonte == 'Recursos de outros candidatos/comitês':
            if nome_doador.find('Comitê') != -1:
                # origem = partido_origem+'-COM-'+registro['Sigla UE doador']
                origem = partido_origem+'-'+registro['Sigla UE doador']
            else:
                origem = partido_origem+'-CAN'
        else:
            origem = partido_origem+'-'+registro['Sigla UE doador']
        
    if doador == '48540421000131':
        print('%s: %s -> %s(%s-%s): %s' % ( 
            registro['Data da receita'][:5], nome_doador,
            registro['Tipo Comite'], partido, registro['UF'], valor))
        # Não iremos diferencia o comitê do diretório estadual para que o gráfico final não fique poluído.
        # if registro['Tipo Comite'].find('Comitê') != -1:
        #    destino = partido+'-COM-'+registro['UF']
        # else:
        destino = partido+'-'+registro['UF']
        add_node(origem,destino,valor)
        print(origem,destino)
        count += 1

19/09: SERVENG CIVILSAN S A EMPRESAS ASSOCIADAS DE ENGENHARIA -> Comitê Financeiro Único(PSDB-RN): 100000.0
PJ PSDB-RN
30/09: SERVENG CIVILSAN S/A EMPRESAS ASSOCIADAS DE ENGENHARIA -> Comitê Financeiro Único(PRB-SP): 200000.0
PJ PRB-SP
19/09: SERVENG CIVILSAN S/A EMPRESAS ASSOSSIADAS DE ENGENHARIA -> Comitê Financeiro Único(PSB-SP): 200000.0
PJ PSB-SP
15/09: SERVENG CIVILSAN S/A EMPRESAS ASSOCIADAS DE ENGENHARIA -> Comitê Financeiro Único(PMDB-DF): 8000.0
PJ PMDB-DF
15/09: Direção Estadual/Distrital (PT-AC) -> Comitê Financeiro Distrital/Estadual para Governador(PT-AC): 44000.0
PT-AC PT-AC
23/07: Direção Estadual/Distrital (PSDB-SP) -> Comitê Financeiro Distrital/Estadual para Governador(PSDB-SP): 1000000.0
PSDB-SP PSDB-SP
18/08: Direção Estadual/Distrital (PSDB-SP) -> Comitê Financeiro Distrital/Estadual para Governador(PSDB-SP): 1000000.0
PSDB-SP PSDB-SP
08/10: Direção Estadual/Distrital (PSDB-SP) -> Comitê Financeiro Distrital/Estadual para Governador(PSDB-SP): 900000.0
PSDB-SP PSDB

In [ ]:
count = 0
response = requests.get('http://www.farmi.pro.br/media/uploads/financiamento/receitas_candidatos_2014_brasil.txt')
for registro in csv.DictReader(io.StringIO(response.text), delimiter=';'):
    count += 1
    partido = registro['Sigla  Partido']
    fonte = registro['Tipo receita']
    valor = float(registro['Valor receita'].replace(',','.'))
    doador = registro['CPF/CNPJ do doador originário'].strip()
    if doador == '#NULO':
        doador = registro['CPF/CNPJ do doador'].strip()


    if doador == '48540421000131':
        # print('%s (%s) -> %s: %s (%s)' % (
        #    registro['Nome do doador'], registro['UF'], 
        #    registro['Nome candidato'], valor, fonte))

        numero_origem = registro['Número partido doador']
        numero_destino = partidos[ partido ]
        if numero_origem != '#NULO' and numero_origem != numero_destino:
            partidos_diferentes += 1
            partido_origem = partidos_numero[ numero_origem ]
        else:
            partido_origem = partido

        if fonte == 'Recursos de partido político':
            if registro['Nome do doador'] == 'Direção Nacional':
                origem = partido_origem+'-BR'
            else:
                origem = partido_origem+'-'+registro['UF']

        elif fonte == 'Recursos de outros candidatos/comitês':
            if registro['Nome do doador'].find('Comitê') != -1:
                # origem = partido_origem+'-COM-'+registro['UF']
                origem = partido_origem+'-'+registro['UF']
            else:
                origem = partido_origem+'-CAN'
        else:
            origem = 'PJ'
            
        destino = partido+'-CAN'
        add_node(origem, destino, valor)
        # if origem != destino:
        #    print('%s > %s' % (origem, destino))

print('Total de registros lidos: %d' % count)

Total de registros lidos: 427489


In [ ]:
# Montando a lista de atores (nós do gráfico)
# Fixando o primeiro como a empresa (PJ)
count_atores = 0
atores = {'PJ': count_atores}
labels = ['PJ']
origem = []
destino = []
values = []
for chave in grafo:
    r = grafo[chave]
    index_origem = atores.get(r['origem'], None)
    if index_origem is None:
        count_atores += 1
        index_origem = count_atores
        atores[r['origem']] = index_origem
        labels.append(r['origem'])

    index_destino = atores.get(r['destino'], None)
    if index_destino is None:
        count_atores += 1
        index_destino = count_atores
        atores[r['destino']] = index_destino
        labels.append(r['destino'])
    
    origem.append( index_origem )
    destino.append( index_destino )
    values.append( int(r['total']) )
    
print('Total de Nós:%d' % (count_atores+1))

Total de Nós:41


In [ ]:
# Por fim, mostramos graficamente a movimentação das doações feitas pela empresa entre os atores

import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 20,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = labels,
      color = "blue"
    ),
    link = dict(
      source = origem,
      target = destino,
      value = values
  ))])

fig.update_layout(title_text="Doações do CNPJ 48.540.421/0001-31 em 2014", font_size=12)
fig.show()